In [9]:
'''
IMPORT ALL DEPENDENCIES
--------------------------
Beautiful Soup: Parse HTML objects from web pages
Pymongo: Read and write to MongoDB
Splinter: Automating browser actions to interact with HTML elements
DateTime: Convert dates
re: Hack to remove 'rd', 'th', 'st' from date strings
'''

from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
from pprint import pprint
import requests
from splinter import Browser
from datetime import datetime
from dateutil.parser import parse
import time
import re
import lxml

In [10]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://socialblade.com/youtube/top/50'
browser.visit(url)

In [11]:
html = browser.html
soup = bs(html, 'lxml')

summary = []
ranking = []
viewcount = []
earnings = []
timeline = []

def solve(s):                                             
    return re.sub(r'(\d)(st|nd|rd|th)', r'\1', s)

youtuber = soup.find_all('div', style='float: right; width: 900px;')
for i in youtuber:
    try:
        link = i.find_all('a')[1:]

        for i in link:
            youtuber_link = i['href']
            browser.click_link_by_href(youtuber_link)
            html = browser.html
            soup = bs(html, 'lxml')

            if 'YouTube search results found' in soup.text:
                browser.back()
            else:
                name = soup.find_all('h1', style='float: left; font-size: 1.4em; font-weight: bold; color:#333; margin: 0px; padding: 0px; margin-right: 5px;')[0].text
                primary_stats = soup.find_all('div', id='YouTubeUserTopInfoBlock')

                for i in primary_stats:
                    uploads = i.find_all('span', style='font-weight: bold;')[0].text.replace(',', '')
                    subscribers = i.find_all('span', style='font-weight: bold;')[1].text.replace(',', '')
                    views = i.find_all('span', style='font-weight: bold;')[2].text.replace(',', '')
                    category = i.find_all('span', style='font-weight: bold;')[4].text.replace(',', '')
                    created = i.find_all('span', style='font-weight: bold;')[5].text.replace(',', '')
                    if created == '--':
                        created = '1900-01-01'
                    else:
                        created = datetime.strptime(solve(created), "%b %d %Y").strftime('%Y-%m-%d')

                    summary.append({'name': name,
                                    'uploads': uploads,
                                    'subscribers': subscribers,
                                    'views': views,
                                    'category': category,
                                    'created': created})

                ranking_stats = soup.find_all('div', style='height: 100px; width: 860px; border-bottom: 1px solid #eee;')
                for i in ranking_stats:
                    grade = i.find_all('p')[0].text
                    subscriber_rank = i.find_all('p')[2].text[:-2].replace(',', '')
                    view_rank = i.find_all('p')[4].text[:-2].replace(',', '')
                    socialblade_rank = i.find_all('p')[6].text[:-2].replace(',', '')

                    ranking.append({'name': name,
                                'grade': grade,
                                'subscriber_rank': subscriber_rank,
                                'view_rank': view_rank,
                                'socialblade_rank': socialblade_rank})

                views_stats = soup.find_all('div', style='height: 70px; width: 860px; border-bottom: 1px solid #eee;')
                for i in views_stats:
                    views_last30d_count = i.find_all('span', id='afd-header-views-30d')[0].text.replace(',', '').replace('\n', '')
                    subs_last30d_count = i.find_all('span', id='afd-header-subs-30d')[0].text.replace(',', '').replace('\n', '')

                    viewcount.append({'name': name,
                                'views_last30d_count': views_last30d_count,
                                'subs_last30d_count': subs_last30d_count})

                earnings_stats = soup.find_all('div', style='height: 80px; width: 860px;')
                for i in earnings_stats:
                    min_monthly_earnings = i.find_all('p', style='font-size: 1.4em; color:#41a200; font-weight: 600; padding-top: 20px;')[0].text.split('-')[0].replace('$', '').replace('.', '').replace('K', '00').replace('M', '00000').strip()
                    max_monthly_earnings = i.find_all('p', style='font-size: 1.4em; color:#41a200; font-weight: 600; padding-top: 20px;')[0].text.split('-')[1].replace('$', '').replace('.', '').replace('K', '00').replace('M', '00000').strip()
                    min_annual_earnings = i.find_all('p', style='font-size: 1.4em; color:#41a200; font-weight: 600; padding-top: 20px;')[1].text.split('-')[0].replace('$', '').replace('.', '').replace('K', '00').replace('M', '00000').strip()
                    max_annual_earnings = i.find_all('p', style='font-size: 1.4em; color:#41a200; font-weight: 600; padding-top: 20px;')[1].text.split('-')[1].replace('$', '').replace('.', '').replace('K', '00').replace('M', '00000').strip()

                    earnings.append({'name': name,
                                'min_monthly_earnings': min_monthly_earnings,
                                'max_monthly_earnings': max_monthly_earnings,
                                'min_annual_earnings': min_annual_earnings,
                                'max_annual_earnings': max_annual_earnings})

                timeline_statsA = soup.find_all('div', style='width: 860px; height: 32px; line-height: 32px; background: #f8f8f8;; padding: 0px 20px; color:#444; font-size: 9pt; border-bottom: 1px solid #eee;')
                for i in timeline_statsA:
                    timeline_dateA = i.find_all('div', style='float: left; width: 95px;')[0].text.replace('\n', '')
                    timeline_subsA = i.find_all('div', style='width: 140px; float: left;')[0].text.replace('\n', '').replace(',', '')
                    timeline_viewsA = i.find_all('div', style='width: 140px; float: left;')[1].text.replace('\n', '').replace(',', '')

                    timeline_min_earningsA = i.find_all('div', style='float: left; width: 165px; height: 30px;')[0].text.split('-')[0].replace('$', '').strip()
                    if '.' in timeline_min_earningsA:
                        timeline_min_earningsA = timeline_min_earningsA.replace('.', '').replace('K', '00').replace('M', '00000')
                    else:
                        timeline_min_earningsA = timeline_min_earningsA.replace('K', '000').replace('M', '000000')

                    timeline_max_earningsA = i.find_all('div', style='float: left; width: 165px; height: 30px;')[0].text.split('-')[1].replace('$', '').strip()
                    if '.' in timeline_max_earningsA:
                        timeline_max_earningsA = timeline_max_earningsA.replace('.', '').replace('K', '00').replace('M', '00000')
                    else:
                        timeline_max_earningsA = timeline_max_earningsA.replace('K', '000').replace('M', '000000')

                    timeline.append({'name': name,
                                    'date': timeline_dateA,
                                   'subscribers': timeline_subsA,
                                    'views': timeline_viewsA,
                                    'min_earnings': timeline_min_earningsA,
                                    'max_earnings': timeline_max_earningsA})

                timeline_statsB = soup.find_all('div', style='width: 860px; height: 32px; line-height: 32px; background: #fcfcfc; padding: 0px 20px; color:#444; font-size: 9pt; border-bottom: 1px solid #eee;')
                for i in timeline_statsB:
                    timeline_dateB = i.find_all('div', style='float: left; width: 95px;')[0].text.replace('\n', '').strip()
                    timeline_subsB = i.find_all('div', style='width: 140px; float: left;')[0].text.replace(',', '').replace('\n', '').strip()
                    timeline_viewsB = i.find_all('div', style='width: 140px; float: left;')[1].text.replace('\n', '').replace(',', '').strip()

                    timeline_min_earningsB = i.find_all('div', style='float: left; width: 165px; height: 30px;')[0].text.split('-')[0].replace('$', '').strip()
                    if '.' in timeline_min_earningsB:
                        timeline_min_earningsB = timeline_min_earningsB.replace('.', '').replace('K', '00').replace('M', '00000')
                    else:
                        timeline_min_earningsB = timeline_min_earningsB.replace('K', '000').replace('M', '000000')

                    timeline_max_earningsB = i.find_all('div', style='float: left; width: 165px; height: 30px;')[0].text.split('-')[1].replace('$', '').strip()
                    if '.' in timeline_max_earningsB:
                        timeline_max_earningsB = timeline_max_earningsB.replace('.', '').replace('K', '00').replace('M', '00000')
                    else:
                        timeline_max_earningsB = timeline_max_earningsB.replace('K', '000').replace('M', '000000')

                    timeline.append({'name': name,
                                    'date': timeline_dateB,
                                    'subscribers': timeline_subsB,
                                    'views': timeline_viewsA,
                                    'min_earnings': timeline_min_earningsB,
                                    'max_earnings': timeline_max_earningsB})


                browser.back()

    except Exception as e:
        print(e)


In [17]:
summary_df = pd.DataFrame(summary)
views_df = pd.DataFrame(viewcount)
ranking_df = pd.DataFrame(ranking)
earnings_df = pd.DataFrame(earnings)
timeline_df = pd.DataFrame(timeline)

summary_df.to_csv('summary_df.csv')
views_df.to_csv('views_df.csv')
ranking_df.to_csv('ranking_df.csv')
earnings_df.to_csv('earnings_df.csv')
timeline_df.to_csv('timeline_df.csv')

In [18]:
views_df

,name,subs_last30d_count,views_last30d_count
0,T-Series,5298100,2473896000
1,SET India,1916537,1365789000
2,5-Minute Crafts,3194580,708033000
3,Zee TV,961489,0
4,VN Children’s Music,0,414468000
5,Cocomelon - Nursery Rhymes,3029725,1264509000
6,Like Nastya Vlog,1821374,0
7,Canal KondZilla,1241395,666564000
8,U2VEVO,-2748,307767000
9,Salih Reis'in Dünyası,0,621600000


In [19]:
ranking_df

,grade,name,socialblade_rank,subscriber_rank,view_rank
0,A++,T-Series,1,5,1
1,A++,SET India,2,11,5
2,A++,5-Minute Crafts,3,8,43
3,A++,Zee TV,4,51,22
4,A++,VN Children’s Music,5,24214500,3490
5,A++,Cocomelon - Nursery Rhymes,7,28,21
6,A++,Like Nastya Vlog,9,186,219
7,A++,Canal KondZilla,10,7,6
8,A+,U2VEVO,13,9356,1620
9,A+,Salih Reis'in Dünyası,11,24408648,278


In [20]:
earnings_df

,max_annual_earnings,max_monthly_earnings,min_annual_earnings,min_monthly_earnings,name
0,118700000,9900000,7400000,618500,T-Series
1,65600000,5500000,4100000,341400,SET India
2,3400000,2800000,2100000,17700,5-Minute Crafts
3,42100000,3500000,2600000,219400,Zee TV
4,19900000,1700000,1200000,103600,VN Children’s Music
5,60700000,5100000,3800000,316100,Cocomelon - Nursery Rhymes
6,32700000,2700000,200000,170500,Like Nastya Vlog
7,3200000,2700000,200000,166600,Canal KondZilla
8,14800000,1200000,923300,76900,U2VEVO
9,29800000,2500000,1900000,155400,Salih Reis'in Dünyası


In [21]:
timeline_df

,date,max_earnings,min_earnings,name,subscribers,views
0,2018-12-02,348700,21800,T-Series,73109515,54645090144
1,2018-12-04,373000,23300,T-Series,73721976,54817406795
2,2018-12-06,377400,23600,T-Series,74208335,54999469306
3,2018-12-08,348300,21800,T-Series,74635865,55177093764
4,2018-12-10,314700,19700,T-Series,74989383,55342803201
5,2018-12-12,396800,24800,T-Series,75284522,55520621661
6,2018-12-14,431500,27000,T-Series,75395083,55739751384
7,2018-12-03,316300,19800,T-Series,73410923,55739751384
8,2018-12-05,350800,21900,T-Series,73982223,55739751384
9,2018-12-07,362200,22600,T-Series,74438170,55739751384
